In [1]:
from random import choice
from itertools import product, combinations
from groundedPL.codificacion import ToPropositionalLogic
from groundedPL.logUtils import LogUtils
from groundedPL.logic_tester import LogicTester

filas = ['fila1', 'fila2', 'fila3']
columnas = ['columna1', 'columna2', 'columna3']

parser = ToPropositionalLogic()

In [2]:
for f, c in product(filas, columnas):
    formula = f'CABALLO_EN({f}, {c})'
    parser.parse(formula)

In [3]:
def regla1():
    casillas = [(x,y) for x in range(3) for y in range(3)]
    tripletas = list(combinations(casillas, 3))
    lista = []
    for t in tripletas:
        c1, c2, c3 = t
        x1, y1 = c1
        x2, y2 = c2
        x3, y3 = c3
        formula_casillas_caballos = f'CABALLO_EN({x1}, {y1}) & CABALLO_EN({x2}, {y2}) & CABALLO_EN({x3}, {y3})'
        otras_casillas = [c for c in casillas if c not in t]
        lista_negs = [parser.to_nltk(f'-CABALLO_EN({x}, {y})') for x, y in otras_casillas]
        formula_no_caballos = LogUtils.Ytoria(lista_negs)
        formula_tres_caballos = f'{formula_casillas_caballos} & {formula_no_caballos}'
        lista.append(parser.to_nltk(formula_tres_caballos))
    return LogUtils.Otoria(lista)

regla = regla1()

In [ ]:
lt = LogicTester()
formula_lp = lt.translation_to_prover(regla)
res = lt.SATsolve(formula_lp)
if res == 'UNSAT':
    print('La fórmula es insatisfacible')
else:
    print('La fórmula es satisfacible')
    modelo = [lt.to_numeric.literal(x) for x in res]
    modelo = [x for x in modelo if lt.to_numeric.solo_atomo(x) in lt.tseitin.atomos] 
    print(f'\nUn modelo es:\n\n\t{modelo}')
    modelo = [lt.to_lp.modelo_lp.decodificar(x) for x in modelo]
    print(f'\nEl modelo decodificado es:\n\n\t{modelo}')

La fórmula es satisfacible

Un modelo es:

	['ă', '-ē', 'ģ', '-ć', '-ė', '-ħ', '-ċ', '-ě', 'ī']

El modelo decodificado es:

	['CABALLO_EN(0, 0)', '-CABALLO_EN(0, 1)', 'CABALLO_EN(0, 2)', '-CABALLO_EN(1, 0)', '-CABALLO_EN(1, 1)', '-CABALLO_EN(1, 2)', '-CABALLO_EN(2, 0)', '-CABALLO_EN(2, 1)', 'CABALLO_EN(2, 2)']


---